In [1]:
import os
import pandas as pd
import requests
import bs4
import lxml
import functions.web_scrapper_functions as scrpr
import mysql.connector
import json
import uuid
import datetime

import logging
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(filename='Parfum AE Web Scrapper Log.log', level=logging.INFO,
format='%(asctime)s:%(levelname)s:%(message)s')

In [2]:
frag_type_dict = scrpr.get_frag_types()
print("Available fragrance types: \n", frag_type_dict.keys())

Available fragrance types: 
 dict_keys(['Body Mist', 'Car Perfume Diffuser', 'Perfume Oils', 'Our Creations', 'Dahn Al Oud', 'Interior Perfume', 'Musk Oil', 'Perfume Reed Diffuser'])


In [150]:
prod_list_dict = {}

for key, val in frag_type_dict.items():
    try:
        print('Currently scrapping ', key)
        prod_list_dict[key] = scrpr.prod_list(val)
        print('Number of products: ', len(prod_list_dict[key]))
    except:
        print("Error in ", key)

Currently scrapping  Body Mist
Number of products:  313
Currently scrapping  Car Perfume Diffuser
Number of products:  5
Currently scrapping  Perfume Oils
Number of products:  362
Currently scrapping  Our Creations
Number of products:  762
Currently scrapping  Dahn Al Oud
Number of products:  16
Currently scrapping  Interior Perfume
Number of products:  294
Currently scrapping  Musk Oil
Number of products:  10
Currently scrapping  Perfume Reed Diffuser
Number of products:  245


In [180]:
r = requests.get("https://www.parfum.ae/oud-cambodi-synthetic/")
s = bs4.BeautifulSoup(r.content, 'lxml')

In [184]:
int(s.select('div.stars__wrap')[0].select('a')[0].text.replace(' reviews',''))

8

In [48]:
# Title
title = s.select('.product-main__title') # For Oud or Our Creations
title = s.select('.title.page-title') # For all others

# Stars
star = 5 - len(s.select('div.stars__wrap')[0].select(".fas.fa-star.graystar"))

# Number of reviews
reviews = int(s.select('div.stars__wrap')[0].select('a')[0].text.replace(' reviews',''))   # For Oud or Our Creations
reviews = int(s.select('div.review-links')[0].select('a')[0].text.replace(' reviews',''))  # For all others

# Price
price = float(s.select('span.product-js-price')[1].text.replace('AED ',''))   # For Oud or Our Creations
price = float(s.select('div.product-price')[0].text.replace('AED ',''))       # For all others

# Hot or not
if len(s.select('.hot__product.product-main__hot')) > 0: x=True

[<div class="title page-title" style="margin-bottom:0px;">Plum Perfect Body Mist 150ml</div>]

In [198]:
def prod_info(prod_list_dict):
    products = {}
    for frag_type, prod_list in prod_list_dict.items():
        for key, val in prod_list.items():
            r = requests.get(val)
            s = bs4.BeautifulSoup(r.content, 'lxml')
            prod_dict = {}
    
            if frag_type in ['Our Creations','Dahn Al Oud']:
                print(key)
                try:
                    prod_dict['name'] = s.select('.product-main__title')[0].text
                except:
                    prod_dict['title'] = None
                    # logging.info(f"Error retrieving title for url: {val}")
    
                try:
                    prod_dict['stars'] = 5 - len(s.select('div.stars__wrap')[0].select(".fas.fa-star.graystar"))
                except:
                    prod_dict['stars'] = None
                    # logging.info(f"Error retrieving stars for url: {val}")
    
                try:
                    prod_dict['reviews'] = int(s.select('div.stars__wrap')[0].select('a')[0].text.replace(' reviews',''))
                except:
                    prod_dict['reviews'] = None
                    # logging.info(f"Error retreiving reviews for url: {val}")
    
                try:
                    prod_dict['price'] = float(s.select('span.product-js-price')[1].text.replace('AED ',''))
                except:
                    prod_dict['price'] = None
                    # logging.info(f"Error retrieving price for url: {val}")
    
                try:
                    if len(s.select('.hot__product.product-main__hot')) > 0: prod_dict['hot']=True
                except:
                    prod_dict['hot'] = False
                    # logging.error(f"Error retrieving hot label for url: {val}")
    
            else:
                print(key)
                try:
                    prod_dict['name'] = s.select('.title.page-title')[0].text
                except:
                    prod_dict['title'] = None
                    # logging.info(f"Error retrieving title for url: {val}")
    
                try:
                    prod_dict['stars'] = 5 - len(s.select('div.stars__wrap')[0].select(".fas.fa-star.graystar"))
                except:
                    prod_dict['stars'] = None
                    # logging.info(f"Error retrieving stars for url: {val}")
    
                try:
                    prod_dict['reviews'] = int(s.select('div.review-links')[0].select('a')[0].text.replace(' reviews',''))
                except:
                    prod_dict['reviews'] = None
                    # logging.info(f"Error retreiving reviews for url: {val}")
    
                try:
                    prod_dict['price'] = float(s.select('div.product-price')[0].text.replace('AED ',''))
                except:
                    prod_dict['price'] = None
                    # logging.info(f"Error retrieving price for url: {val}")
    
                try:
                    if len(s.select('.hot__product.product-main__hot')) > 0: prod_dict['hot']=True
                except:
                    prod_dict['hot'] = False
                    # logging.error(f"Error retrieving hot label for url: {val}")
    
            products[key] = prod_dict
    
    return products

In [199]:
products = prod_info(prod_list_dict)

Oud Cambodi Synthetic
Oud Suyufi Synthetic
Zafran Oud Synthetic
Oud Hindi Sweety Synthetic
Black Oud Synthetic
Oud Bormi Synthetic
Oud Khalifa Synthetic
Oud Assami Synthetic
Oud Khas Synthetic
White Oud Synthetic
Oud Trad Honey Synthetic
White Oud Super Synthetic
Oud Francy Synthetic
Oud Saudi Synthetic
Bulgari Gulab Synthetic
Oud Turabi Synthetic
Wild Deer Musk Oil (12ml or 1 Tola)
Tahara Musk Oil (12ml or 1 Tola)
Vanilla Musk Oil (12ml or 1 Tola)
Oud Musk Oil (12ml or 1 Tola)
Pears Musk Oil (12ml or 1 Tola)
Peach Musk Oil (12ml or 1 Tola)
Cherry Musk Oil (12ml or 1 Tola)
Pomegranate Musk Oil (12ml or 1 Tola)
Lavender Musk Oil (12ml or 1 Tola)
Rose Musk Oil (12ml or 1 Tola)


In [200]:
products

{'Oud Cambodi Synthetic': {'name': 'Oud Cambodi Synthetic',
  'stars': 5,
  'reviews': 8,
  'price': 120.0},
 'Oud Suyufi Synthetic': {'name': 'Oud Suyufi Synthetic',
  'stars': 5,
  'reviews': 2,
  'price': 120.0},
 'Zafran Oud Synthetic': {'name': 'Zafran Oud Synthetic',
  'stars': 5,
  'reviews': 1,
  'price': 55.0},
 'Oud Hindi Sweety Synthetic': {'name': 'Oud Hindi Sweety Synthetic',
  'stars': 5,
  'reviews': 1,
  'price': 120.0},
 'Black Oud Synthetic': {'name': 'Black Oud Synthetic',
  'stars': 5,
  'reviews': None,
  'price': 100.0},
 'Oud Bormi Synthetic': {'name': 'Oud Bormi Synthetic',
  'stars': 5,
  'reviews': None,
  'price': 130.0},
 'Oud Khalifa Synthetic': {'name': 'Oud Khalifa Synthetic',
  'stars': 5,
  'reviews': None,
  'price': 120.0},
 'Oud Assami Synthetic': {'name': 'Oud Assami Synthetic',
  'stars': 5,
  'reviews': None,
  'price': 110.0},
 'Oud Khas Synthetic': {'name': 'Oud Khas Synthetic',
  'stars': 5,
  'reviews': None,
  'price': 150.0},
 'White Oud Syn

In [191]:
for key, val in prod_list_dict.items():
    print(key in ['Our Creations','Dahn Al Oud'])

True
False


In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the web driver (make sure you have the appropriate driver installed)
driver = webdriver.Chrome()
url = "https://www.parfum.ae/lv-imagination-body-mist/"
driver.get(url)

# Click the tab you want (e.g., "Specification" tab)
tab_selector = driver.find_element(By.ID, "tab-review-tab")
tab_selector.click()

# Wait for the dynamic content to load (adjust the timeout as needed)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.ID, "tab-review")))

# Extract data from the selected tab
page_content = driver.page_source
soup = BeautifulSoup(page_content, "lxml")

# Now, you can use BeautifulSoup to parse the content of the selected tab
# For example: data = soup.find("element selector").text

# Close the web driver when you're done
driver.quit()


In [41]:
soup.select("div#review")[0].select("p")

[<p>Omg! I’m glad I bought this perfume! It actually smells like the LV imagination. I have the original LV but don’t want to use it all the time as it’s quiet pricey. So thanks God for the this dupe! Definitely will order more. Highly recommended </p>]

In [ ]:
conn = mysql.connector.connect(
    host = "localhost",
    user = os.environ.get("MYSQL_USERNAME"),
    password = os.environ.get("MYSQL_PASSWORD"),
    database = "parfumae_products"
)

cursor = conn.cursor()

In [ ]:

try:
    for key, val in frag_type_dict.items():
        serialized_value = json.dumps(val)

        insert_query = "INSERT INTO fragrance_types (name, url) VALUES (%s, %s)"
        cursor.execute(insert_query, (key, serialized_value))
    
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()

cursor.close()
conn.close()